In [422]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from datetime import datetime
import numpy as np
#import tensorflow as tf

#  Import and read the file.
supply_df = pd.read_csv("C:/Users/Elissa/UCB/dataVizfinal/SupplyData/CAISO-ALL-SUPPLY-DATA2.csv")
supply_df.head()


,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,4/10/2018,0:00:00,2018-04-10 00:00:00,1.523318e+09
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,4/10/2018,0:05:00,2018-04-10 00:05:00,1.523319e+09
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,4/10/2018,0:10:00,2018-04-10 00:10:00,1.523319e+09
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,4/10/2018,0:15:00,2018-04-10 00:15:00,1.523319e+09
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,4/10/2018,0:20:00,2018-04-10 00:20:00,1.523320e+09


In [423]:
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Large hydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Natural gas']


In [424]:
supply_df.dtypes

Renewables        float64
Natural gas       float64
Large hydro       float64
Imports           float64
Batteries         float64
Nuclear           float64
Coal              float64
Other             float64
Date               object
Time               object
DateTime           object
Unix Timestamp    float64
Sum               float64
dtype: object

In [425]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

         0   1   2
0        0  00  00
1        0  05  00
2        0  10  00
3        0  15  00
4        0  20  00
...     ..  ..  ..
284793  23  35  00
284794  23  40  00
284795  23  45  00
284796  23  50  00
284797  23  55  00

[284798 rows x 3 columns]


In [426]:
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)

In [427]:
baby_df= baby_df.drop([2],1)

In [428]:
baby_df.dtypes

0    int32
1    int32
dtype: object

In [429]:
# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})

In [430]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year
#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)

baby_df.head(100)

,Hours,Minutes,Year
0,0,0,2018
1,0,5,2018
2,0,10,2018
3,0,15,2018
4,0,20,2018
...,...,...,...
95,7,55,2018
96,8,0,2018
97,8,5,2018
98,8,10,2018


In [431]:
#Make a date string column 
date_df = pd.DataFrame()
supply_df['Date1']=supply_df['Date'].astype(str)

date_df = supply_df['Date1'].str.split("-", expand=True)
date_df['DateAttribute'] = date_df[1] + date_df[2]
date_df = date_df.drop([0,1,2],1)


In [432]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [433]:
calc_df = pd.merge(baby_df, date_df, left_index=True, right_index=True, how='outer')
calc_df.head(-5)

,Hours,Minutes,Year,DemandMW,DateAttribute
0,0,0,2018,6.2356,0410
1,0,5,2018,6.2356,0410
2,0,10,2018,6.2356,0410
3,0,15,2018,6.2356,0410
4,0,20,2018,6.2356,0410
...,...,...,...,...,...
284788,23,10,2020,9.9174,1231
284789,23,15,2020,9.9174,1231
284790,23,20,2020,9.9174,1231
284791,23,25,2020,9.9174,1231


In [434]:
calc_df.dtypes


Hours              int32
Minutes            int32
Year               int32
DemandMW         float64
DateAttribute     object
dtype: object

In [435]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [436]:
supply_df['Date'] = supply_df['Date'].apply(fun)
supply_df.head(1300)

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp,Sum,Date1
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,100,0:00:00,2018-04-10 00:00:00,1.523318e+09,22879.0,2018-04-10
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,100,0:05:00,2018-04-10 00:05:00,1.523319e+09,22794.0,2018-04-10
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,100,0:10:00,2018-04-10 00:10:00,1.523319e+09,22743.0,2018-04-10
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,100,0:15:00,2018-04-10 00:15:00,1.523319e+09,22488.0,2018-04-10
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,100,0:20:00,2018-04-10 00:20:00,1.523320e+09,22368.0,2018-04-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,3262.0,4010.0,3096.0,5808.0,-3.0,2253.0,7.0,0.0,105,4:15:00,2018-04-15 04:15:00,1.523766e+09,18433.0,2018-04-15
1296,3206.0,4027.0,3102.0,5786.0,0.0,2253.0,7.0,0.0,105,4:20:00,2018-04-15 04:20:00,1.523766e+09,18381.0,2018-04-15
1297,3187.0,4076.0,3106.0,5783.0,0.0,2252.0,7.0,0.0,105,4:25:00,2018-04-15 04:25:00,1.523766e+09,18411.0,2018-04-15
1298,3144.0,4018.0,3103.0,5879.0,0.0,2253.0,7.0,0.0,105,4:30:00,2018-04-15 04:30:00,1.523767e+09,18404.0,2018-04-15


In [437]:
supply_df = supply_df.drop(['Date1'],1)
final_df = pd.merge(supply_df, calc_df, left_index=True, right_index=True, how='outer')
final_df

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp,Sum,Hours,Minutes,Year,DemandMW,DateAttribute
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,100,0:00:00,2018-04-10 00:00:00,1.523318e+09,22879.0,0,0,2018,6.2356,0410
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,100,0:05:00,2018-04-10 00:05:00,1.523319e+09,22794.0,0,5,2018,6.2356,0410
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,100,0:10:00,2018-04-10 00:10:00,1.523319e+09,22743.0,0,10,2018,6.2356,0410
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,100,0:15:00,2018-04-10 00:15:00,1.523319e+09,22488.0,0,15,2018,6.2356,0410
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,100,0:20:00,2018-04-10 00:20:00,1.523320e+09,22368.0,0,20,2018,6.2356,0410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284793,3183.0,8540.0,764.0,8800.0,122.0,1144.0,12.0,0.0,366,23:35:00,2020-12-31 23:35:00,1.609458e+09,22565.0,23,35,2020,9.9174,1231
284794,3294.0,8450.0,718.0,8804.0,99.0,1144.0,12.0,0.0,366,23:40:00,2020-12-31 23:40:00,1.609458e+09,22521.0,23,40,2020,9.9174,1231
284795,3410.0,8300.0,731.0,8725.0,90.0,1143.0,13.0,0.0,366,23:45:00,2020-12-31 23:45:00,1.609458e+09,22412.0,23,45,2020,9.9174,1231
284796,3456.0,8217.0,753.0,8621.0,91.0,1144.0,12.0,0.0,366,23:50:00,2020-12-31 23:50:00,1.609459e+09,22294.0,23,50,2020,9.9174,1231


In [438]:
df1= pd.DataFrame()

In [439]:
df1 = final_df.groupby(['DateAttribute','Time'])['Sum'].agg('mean')
df1 = df1.reset_index(level=None)
df1 = df1.rename(columns={df1.columns[2]: 'AvgSUM(3yr)'})

print(len(df1))
df1

105408


,DateAttribute,Time,AvgSUM(3yr)
0,0101,0:00:00,22735.500000
1,0101,0:05:00,22789.000000
2,0101,0:10:00,22778.000000
3,0101,0:15:00,22732.000000
4,0101,0:20:00,22691.500000
...,...,...,...
105403,1231,9:35:00,23848.333333
105404,1231,9:40:00,23814.666667
105405,1231,9:45:00,23768.333333
105406,1231,9:50:00,23686.000000


In [444]:
df1= df1.sort_values(['DateAttribute','AvgSum(3yr)', ascending[True,False],inplace=True)
df1

SyntaxError: invalid syntax (<ipython-input-444-d3731dff9ff7>, line 1)

In [ ]:
x = final_df.loc[(final_df['DateAttribute'] == '2-29')]
print(x.index)
y = df1.loc[(final_df['DateAttribute'] == '2-29')]
print(y.index)

In [ ]:
final_df = final_df.drop(final_df.index[197257:197545])
final_df = final_df.reset_index(level=None)

In [ ]:
print(type(final_df))
print(final_df.loc[197256:197560])

In [ ]:
print(len(final_df))
final_df.iloc[-1]

In [ ]:
copy1_df1 = df1.copy()
copy2_df1 = df1.copy()


In [ ]:
new_df1 = df1.append(copy1_df1)

In [ ]:
new_df2 = new_df1.append(copy2_df1)
print(len(new_df2))